In [1]:
import sys
sys.path.append("../../")
import fibi.src.pie_chart as pie_chart, fibi.src.statistical_test as statistical_test
import pathlib as p
import numpy as np
import tqdm
import json
from collections import defaultdict
import pandas as pd
import pyarrow.parquet as pq
root = p.Path("../data/algorithms_out/tsp/data_TSP_Uniformly distr. towns_quad_3opt.parquet").resolve()
root_folder = p.Path("../").resolve()
table = pq.read_table(root)
df = table.to_pandas()
df.head()

,num_iter,num_iter_glob,init_cost,final_cost,duration,SEED_GLOB,SEED_PROBLEM,SEED_ASSIGN,DATASET,FI_BI,IMPR,NUM_TOWNS,NUM_DIM,OPT,filename,KEY,PROBLEM,init_meth,ratio
0,11.0,43.0,878.90566,364.06950,0.003344,0.0,0.0,0.0,Uniformly distr. towns,BI,RAND,20,2.0,3.0,rand_3opt,0.0,TSP,random,0.41423
1,18.0,86.0,1532.47995,434.84915,0.011795,1.0,0.0,0.0,Uniformly distr. towns,BI,RAND,30,2.0,3.0,rand_3opt,1.0,TSP,random,0.28376
2,27.0,167.0,1967.04181,492.93251,0.034221,2.0,0.0,0.0,Uniformly distr. towns,BI,RAND,40,2.0,3.0,rand_3opt,2.0,TSP,random,0.25060
3,31.0,226.0,2563.46930,538.55466,0.103742,3.0,0.0,0.0,Uniformly distr. towns,BI,RAND,50,2.0,3.0,rand_3opt,3.0,TSP,random,0.21009
4,34.0,303.0,3088.83334,607.23457,0.175159,4.0,0.0,0.0,Uniformly distr. towns,BI,RAND,60,2.0,3.0,rand_3opt,4.0,TSP,random,0.19659


In [2]:
list_cases_per_init = {}
allowed_init = {"RAND", "GREEDY", "GREEDY randomized"}
for initialization, df_init in df.groupby("IMPR"):
    if initialization not in allowed_init:
        continue
    L_cases = []
    for instance, df_instance in df_init.groupby(["NUM_TOWNS"]):
        df_instance.sort_values(by="SEED_GLOB", inplace=True)
        bi = df_instance.query("FI_BI == 'BI'")["final_cost"].to_numpy()
        fi = df_instance.query("FI_BI == 'FI'")["final_cost"].to_numpy()
        init = df_instance.query("FI_BI == 'FI'")["init_cost"].to_numpy()
        diff = (bi-fi)/init
        case = pie_chart.get_case_from_diff(
            diff=diff,
            maximization=False,
            init_random=initialization == "RAND"
        )
        L_cases.append(case)

    assert len(L_cases) > 0, "More than 0 samples needed"
    # We build the input for the pie chart function (one list asssociated to the initialization random)
    list_cases_per_init[initialization] = L_cases

In [3]:

results = []
for f in p.Path("../data/algorithms_out/tsp/3opt_greedyrandomized/").rglob("*.txt"):
    with open(f) as fp:
        for l in fp.readlines():
            l = l.strip()[:-1]
            try:
                l = json.loads(l)
            except Exception as e:
                print(l)
                raise e
            results.append(l)
df = pd.DataFrame(results)


In [4]:
def find_contiguous_intervals(nums):
    if not nums:
        return []

    # Step 1: Sort the list of integers
    nums.sort()

    # Step 2: Initialize the list to hold intervals
    intervals = []

    # Step 3: Initialize the start and end of the first interval
    start = nums[0]
    end = nums[0]

    # Step 4: Iterate through the sorted list
    for num in nums[1:]:
        if num == end + 1:
            # If the current number is contiguous, extend the interval
            end = num
        else:
            # If it's not contiguous, finalize the current interval and start a new one
            intervals.append((start, end))
            start = num
            end = num

    # Don't forget to append the last interval
    intervals.append((start, end))

    return intervals

In [11]:
tmp = """#!/bin/bash
#SBATCH --output=log-%x-%j.out
#SBATCH --error=log-%x-%j.err
#SBATCH --nodes=1
#SBATCH --ntasks-per-node=1
#SBATCH --cpus-per-task=16
#SBATCH --time=23:59:00
#SBATCH --mem=8G
# Capture the start time
module load gcc/12.3

start_time=$$(date +"%Y-%m-%d %H:%M:%S")
/scratch/rmoine/FIBI -path_out /scratch/rmoine/dataset_3opt$id.txt -root_data /scratch/rmoine/folder/ -dataset 0 -n_treads 16 -impr 1 -problem tsp -num_rep 1000 -opt 3 -seed_start $start -seed_end $end
# Capture the end time
end_time=$$(date +"%Y-%m-%d %H:%M:%S")

# Calculate and display the execution time
start_seconds=$$(date -d "$$start_time" '+%s')
end_seconds=$$(date -d "$$end_time" '+%s')
execution_time=$$((end_seconds - start_seconds))

echo "Execution time: $$execution_time seconds" """

import string, pyperclip
tmp = string.Template(tmp)
seeds = list(set(range(52000))-set(df["SEED_GLOB"].unique()))
seeds = find_contiguous_intervals(seeds)
# seeds = np.linspace(0,52000,30).astype(int)
# seeds[-1] = 54000
folder = p.Path("./launch/")
folder.mkdir(exist_ok=True)
command = []
for i, (i1, i2) in enumerate(seeds):
    j = i + 100
    s = tmp.substitute(id=str(j),start=int(i1), end=int(i2))
    name = f"launch_3opt_{j}"
    with open(folder / name, "w") as fp:
        fp.write(s)
    command.append(f"sb {name};")
pyperclip.copy(" ".join(command))

In [5]:

initialization = "GREEDY randomized"
L_cases = []
for instance, df_instance in df.groupby(["NUM_TOWNS"]):
    df_instance.sort_values(by="SEED_GLOB", inplace=True)
    bi = df_instance.query("FI_BI == 0")["final_cost"].to_numpy()
    fi = df_instance.query("FI_BI == 1")["final_cost"].to_numpy()
    init = df_instance.query("FI_BI == 0")["init_cost"].to_numpy()
    print(len(bi),len(fi))
    diff = (bi-fi)/init
    case = pie_chart.get_case_from_diff(
        diff=diff,
        maximization=False,
        init_random=initialization == "RAND"
    )
    L_cases.append(case)
list_cases_per_init[initialization] = L_cases

1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000
1000 1000


In [6]:

problem = "tsp"
dataset = "randomUniform3opt"
# We create the piechart
figure_text = pie_chart.make_latex_piecharts_figure_for_datasets_of_problem(
    list_cases_per_init=list_cases_per_init,
    problem=problem,
    dataset=dataset,
    template_folder=root_folder / "data" / "templates" / "latex"
)
# And save it
with open(root_folder / "data" / "out_diagrams" / f"{problem}_{dataset}.tex", "w", encoding="utf-8") as fp:
    fp.write(figure_text)